In [1]:
!pip install sparknlp


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, broadcast, current_timestamp, regexp_replace
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, TimestampType
import sparknlp

# Inicialización de SPARK

In [3]:
# --- IMPORTS Y CONFIGURACIÓN ---
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import sparknlp 
# Importa el resto de las funciones necesarias para tu pipeline de NLP

# --- CONFIGURACIÓN DE PAQUETES ---
SPARK_MASTER = "spark://0.0.0.0:7077" # Dirección de tu Spark Master dentro de Docker
SPARK_NLP_VERSION = "5.3.3" 

# Lista de paquetes para Spark (Kafka, Delta, y Spark NLP)
SPARK_PACKAGES = (
    "io.delta:delta-spark_2.12:3.1.0,"
    "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1,"
    f"com.johnsnowlabs.nlp:spark-nlp_2.12:{SPARK_NLP_VERSION}" 
)

# --- INICIALIZACIÓN DE SPARK CON PAQUETES ---


# --- CREACIÓN DE LA SPARK SESSION ---
spark = (SparkSession.builder \
    .appName("GoldLayer_BERT_Embeddings") \
    .master(SPARK_MASTER) \
    .config("spark.jars.packages", SPARK_PACKAGES) \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    

    .config("spark.driver.memory", "8g") \
    
    .config("spark.executor.memory", "4g") \
    
    .getOrCreate())

print(f"✅ Sesión de Spark iniciada y conectada al Master {SPARK_MASTER} con Spark NLP v{SPARK_NLP_VERSION}.")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ffab3b4a-7013-456c-9c9e-efbd363289d9;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in ce

✅ Sesión de Spark iniciada y conectada al Master spark://0.0.0.0:7077 con Spark NLP v5.3.3.


# FASE 2

# Ingesta: Leer stream de Kafka

In [4]:
# Esquema de los datos reales del productor (8 campos clave + 6 auxiliares)
CONTRACT_SCHEMA = StructType([
    # Campos clave para ML
    StructField("id_contrato", StringType(), True),
    StructField("objeto_contrato", StringType(), True),
    StructField("entidad", StringType(), True),
    StructField("codigo_unspsc", StringType(), True), 
    StructField("duracion_dias", LongType(), True),
    StructField("valor_contrato", DoubleType(), True),
    StructField("fecha_firma", StringType(), True),
    StructField("departamento", StringType(), True),
    
    # Columnas ruidosas / auxiliares (Serán eliminadas)
    StructField("nit_entidad", StringType(), True), 
    StructField("localizacion", StringType(), True),
    StructField("sector", StringType(), True), 
    StructField("es_pyme", StringType(), True),
    StructField("valor_facturado", StringType(), True), # Aunque es numérico, puede venir sucio
    StructField("urlproceso", StringType(), True),
])

# 1. Tarea: Leer el stream de Kafka
kafka_stream = (spark.readStream
    .format("kafka") 
    .option("kafka.bootstrap.servers", "kafka:29092")
    .option("subscribe", "contratos-publicos")
    .option("startingOffsets", "earliest") # Para procesar datos desde el inicio (si el productor ya corrió)
    .load()
)

print("Stream de Kafka configurado.")

Stream de Kafka configurado.


# Preparación de los datos

In [5]:
# Definimos los departamentos que forman parte del Eje Cafetero para el filtro
regiones_eje_cafetero = [
    ("Antioquia", "Eje Cafetero"), ("Caldas", "Eje Cafetero"), 
    ("Quindio", "Eje Cafetero"), ("Risaralda", "Eje Cafetero"), 
    ("Tolima", "Eje Cafetero"), ("Valle del Cauca", "Eje Cafetero")
]
df_regiones = spark.createDataFrame(regiones_eje_cafetero).toDF("departamento_join", "macrorregion_turistica")

# Aplicamos Broadcasting para optimizar el JOIN (Broadcasting Join)
df_regiones_broadcast = broadcast(df_regiones)
print("DataFrame de Regiones (Broadcast) listo para el Join.")

DataFrame de Regiones (Broadcast) listo para el Join.


# Persistencia en Delta-Lake

In [6]:
df_silver = (kafka_stream \
    # 2. Explosión de Metadatos
    .withColumn("value_content", from_json(col("value").cast("string"), CONTRACT_SCHEMA)) \
    .select(
        col("value_content.*"),
        col("timestamp").alias("kafka_ingestion_time"), # Metadato de Kafka
        col("offset").alias("kafka_offset")             # Metadato de Kafka
    ) \
    
    # 3. Tarea: Limpieza (Eliminación de Redundantes)
    .drop("nit_entidad", "localizacion", "sector", "es_pyme", "valor_facturado", "urlproceso") \
    
    # 3. Tarea: Cruce con Regiones (Broadcasting Join)
    .join(
        df_regiones_broadcast,
        on=df_regiones_broadcast.departamento_join == col("departamento"),
        how="inner" # INNER JOIN garantiza que solo pasen los del Eje Cafetero
    ) \
    .drop("departamento_join") \
    
    # Limpieza final de valores (solo si es necesario para asegurar tipos, aunque ya se hizo en el productor)
    .withColumn("departamento", col("departamento").cast(StringType())) \
    .withColumn("processing_time", current_timestamp())
)

In [7]:
# Rutas para el almacenamiento Delta Lake
DELTA_LAKE_PATH = "/opt/spark/data/delta/silver_contracts"
CHECKPOINT_PATH = "/opt/spark/data/checkpoints/silver_contracts"

# Persistencia Delta-Lake

In [8]:
# Rutas para el almacenamiento Delta Lake

# 4. Tarea: Guardar los datos limpios en formato Delta Lake
query = (df_silver.writeStream \
    .format("delta") \
    .outputMode("append") # Añadir nuevos registros
    .option("checkpointLocation", CHECKPOINT_PATH) # Obligatorio para Spark Streaming
    .option("path", DELTA_LAKE_PATH) 
    .trigger(processingTime='10 seconds') # Procesa nuevos datos cada 10 segundos
    .start()
)

print(f"Escritura del Stream a Delta Lake iniciada. Estado del Query ID: {query.id}")
print("El Job está corriendo. Presiona el botón de 'Stop' o interrupción del kernel en Jupyter para detenerlo.")

Escritura del Stream a Delta Lake iniciada. Estado del Query ID: 2fa62f54-f0c6-4035-95cb-0beec4f42da7
El Job está corriendo. Presiona el botón de 'Stop' o interrupción del kernel en Jupyter para detenerlo.


25/12/07 01:27:01 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


# FASE 3

## Embeddings BERT

In [9]:
# EJECUTA ESTO PRIMERO para limpiar la tabla anterior
import shutil
import os

GOLD_DELTA_PATH = "/opt/spark/data/delta/gold_contracts"
GOLD_CHECKPOINT_PATH = "/opt/spark/data/checkpoints/gold_contracts"

# Detener el query anterior si está corriendo
try:
    spark.streams.get("df023a71-e21f-4108-ad66-dada639a0ce4").stop()
except:
    pass

# Eliminar los datos y checkpoints anteriores
if os.path.exists(GOLD_DELTA_PATH):
    shutil.rmtree(GOLD_DELTA_PATH)
    print(f"✅ Eliminado {GOLD_DELTA_PATH}")

if os.path.exists(GOLD_CHECKPOINT_PATH):
    shutil.rmtree(GOLD_CHECKPOINT_PATH)
    print(f"✅ Eliminado {GOLD_CHECKPOINT_PATH}")

print("🔄 Listo para reiniciar el proceso GOLD con el schema correcto.")

✅ Eliminado /opt/spark/data/delta/gold_contracts
✅ Eliminado /opt/spark/data/checkpoints/gold_contracts
🔄 Listo para reiniciar el proceso GOLD con el schema correcto.


In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import ArrayType, FloatType
import sparknlp 
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import BertSentenceEmbeddings
from pyspark.ml import Pipeline

# --- PIPELINE SIMPLIFICADO (SIN FINISHER) ---

document_assembler = DocumentAssembler() \
    .setInputCol("objeto_contrato") \
    .setOutputCol("document")

bert_embeddings = BertSentenceEmbeddings.pretrained("sent_small_bert_L2_128", "en") \
    .setInputCols(["document"]) \
    .setOutputCol("bert_vector") \
    .setMaxSentenceLength(128)

nlp_pipeline = Pipeline(stages=[
    document_assembler,
    bert_embeddings
])

print("✅ Pipeline definido.")

# --- PRE-ENTRENAMIENTO ---
print("⏳ Cargando modelo BERT...")
SILVER_DELTA_PATH = "/opt/spark/data/delta/silver_contracts"
df_sample = spark.read.format("delta").load(SILVER_DELTA_PATH).limit(1)
pipeline_model = nlp_pipeline.fit(df_sample)
print("✅ Modelo BERT cargado.")

# ==============================================================================
# UDF para extraer embeddings
# ==============================================================================

def extract_embeddings(bert_vector_column):
    if bert_vector_column and len(bert_vector_column) > 0:
        first_annotation = bert_vector_column[0]
        if first_annotation and hasattr(first_annotation, 'embeddings'):
            return first_annotation.embeddings
    return None

extract_embeddings_udf = udf(extract_embeddings, ArrayType(FloatType()))

# ==============================================================================
# PROCESAMIENTO STREAMING
# ==============================================================================

GOLD_DELTA_PATH = "/opt/spark/data/delta/gold_contracts"
GOLD_CHECKPOINT_PATH = "/opt/spark/data/checkpoints/gold_contracts"

df_silver_stream = spark.readStream \
    .format("delta") \
    .load(SILVER_DELTA_PATH)

df_gold = pipeline_model.transform(df_silver_stream)

df_gold_final = df_gold.select(
    col("id_contrato"),
    col("entidad"),
    col("departamento"),
    col("valor_contrato"),
    col("duracion_dias"),
    col("codigo_unspsc"),
    extract_embeddings_udf(col("bert_vector")).alias("objeto_embedding"),
    col("kafka_ingestion_time"),
    col("processing_time")
)

query_gold = (df_gold_final.writeStream \
    .format("delta") \
    .outputMode("append") 
    .option("checkpointLocation", GOLD_CHECKPOINT_PATH)
    .option("path", GOLD_DELTA_PATH) 
    .trigger(processingTime='30 seconds')
    .start())

print(f"✅ Proceso GOLD iniciado. Escribiendo en {GOLD_DELTA_PATH}")
print(f"Query ID: {query_gold.id}")

sent_small_bert_L2_128 download started this may take some time.


25/12/07 01:27:02 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Approximate size to download 16.1 MB
[ | ]sent_small_bert_L2_128 download started this may take some time.
Approximate size to download 16.1 MB


25/12/07 01:27:08 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


Download done! Loading the resource.
[ / ]

25/12/07 01:27:10 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
2025-12-07 01:27:10.159699: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-07 01:27:10.264865: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 15627264 exceeds 10% of free system memory.


[OK!]
✅ Pipeline definido.
⏳ Cargando modelo BERT...
✅ Modelo BERT cargado.
✅ Proceso GOLD iniciado. Escribiendo en /opt/spark/data/delta/gold_contracts
Query ID: 63988f95-d303-40ae-9d09-fdc3994e1dfa


25/12/07 01:27:12 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


### Verificación GOLD

In [12]:
from pyspark.sql.functions import slice, size

# Cargar GOLD
df_gold_check = spark.read.format("delta").load(GOLD_DELTA_PATH)

print("=== SCHEMA GOLD ===")
df_gold_check.printSchema()

print("\n=== SAMPLE ROWS (10) ===")
df_gold_check.show(10, truncate=False)

print("\n=== VECTOR SIZE (primeras 10 filas) ")
df_gold_check.select(
    "id_contrato",
    size("objeto_embedding").alias("vector_size")
).show(10, truncate=False)

print("\n=== PRIMEROS VALORES DEL VECTOR")
df_gold_check.select(
    "id_contrato",
    slice("objeto_embedding", 1, 10).alias("primeros_valores")
).show(10, truncate=False)


=== SCHEMA GOLD ===
root
 |-- id_contrato: string (nullable = true)
 |-- entidad: string (nullable = true)
 |-- departamento: string (nullable = true)
 |-- valor_contrato: double (nullable = true)
 |-- duracion_dias: long (nullable = true)
 |-- codigo_unspsc: string (nullable = true)
 |-- objeto_embedding: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- kafka_ingestion_time: timestamp (nullable = true)
 |-- processing_time: timestamp (nullable = true)


=== SAMPLE ROWS (10) ===
+------------------+----------------------------------------------------------------------------+---------------+--------------+-------------+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Variables Categóricas

In [16]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import ArrayType, FloatType
from pyspark.ml.linalg import DenseVector, VectorUDT

from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline

# 1. Leer GOLD
GOLD_DELTA_PATH = "/opt/spark/data/delta/gold_contracts"
df_gold_base = spark.read.format("delta").load(GOLD_DELTA_PATH)

# =====================================================
# A. Convertir array<float> → DenseVector (obligatorio)
# =====================================================

def to_dense_vector(arr):
    if arr is None:
        return None
    return DenseVector(arr)

to_dense_udf = udf(to_dense_vector, VectorUDT())

df_gold_vec = df_gold_base.withColumn(
    "objeto_embedding_vec",
    to_dense_udf(col("objeto_embedding"))
)

# =====================================================
# B. StringIndexer
# =====================================================

indexer_entidad = StringIndexer(
    inputCol="entidad",
    outputCol="entidad_indexed",
    handleInvalid="keep"
)

indexer_unspsc = StringIndexer(
    inputCol="codigo_unspsc",
    outputCol="unspsc_indexed",
    handleInvalid="keep"
)

# =====================================================
# C. OneHotEncoder
# =====================================================

encoder_entidad = OneHotEncoder(
    inputCol="entidad_indexed",
    outputCol="entidad_encoded"
)

encoder_unspsc = OneHotEncoder(
    inputCol="unspsc_indexed",
    outputCol="unspsc_encoded"
)

# =====================================================
# D. VectorAssembler (incluye embeddings + numéricas)
# =====================================================

feature_cols = [
    "objeto_embedding_vec",   # ← EL BUENO
    "entidad_encoded",
    "unspsc_encoded",
    "valor_contrato",
    "duracion_dias"
]

assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features_final"
)

# =====================================================
# E. Pipeline final
# =====================================================

ml_pipeline = Pipeline(stages=[
    indexer_entidad,
    encoder_entidad,
    indexer_unspsc,
    encoder_unspsc,
    assembler
])

print("✅ Pipeline ML completo definido correctamente.")


✅ Pipeline ML completo definido correctamente.


In [17]:
ml_pipeline_model = ml_pipeline.fit(df_gold_vec)
df_ml_ready = ml_pipeline_model.transform(df_gold_vec)


# ----------------------------------------------------
# VERIFICACIÓN DE RESULTADOS
# ----------------------------------------------------
print("\n--- ESQUEMA FINAL CON LAS NUEVAS FEATURES ---")
# Busca las columnas 'entidad_indexed', 'entidad_encoded' y 'features_final'
df_ml_ready.printSchema()

df_ml_ready.select(
    "id_contrato",
    "features_final"
).show(5, truncate=False)



--- ESQUEMA FINAL CON LAS NUEVAS FEATURES ---
root
 |-- id_contrato: string (nullable = true)
 |-- entidad: string (nullable = true)
 |-- departamento: string (nullable = true)
 |-- valor_contrato: double (nullable = true)
 |-- duracion_dias: long (nullable = true)
 |-- codigo_unspsc: string (nullable = true)
 |-- objeto_embedding: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- kafka_ingestion_time: timestamp (nullable = true)
 |-- processing_time: timestamp (nullable = true)
 |-- objeto_embedding_vec: vector (nullable = true)
 |-- entidad_indexed: double (nullable = false)
 |-- entidad_encoded: vector (nullable = true)
 |-- unspsc_indexed: double (nullable = false)
 |-- unspsc_encoded: vector (nullable = true)
 |-- features_final: vector (nullable = true)



+------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------